In [ ]:
%pylab inline
import librosa
from IPython.display import display,Audio
from numpy.fft import fft, ifft

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import sys
import numpy as np
import soundfile as sf

## Analysis
Windowing refers to taking a small frame out of a large signal (you are only looking at your signal through a small window of time). The windowing process alters the spectrum of the signal and this effect should be minimized.

##Processing

Applying a FFT of length N results into N frequency bins starting from 0 up to  with an interval of  where is the sampling rate frequency.
## and Synthesis
So now we managed to adjust the phase in the frequency domain for our current frame. We need to come back to the time domain. The inverse discrete Fourier transform (IDFT) is performed on each frame spectrum. 

In [ ]:
def load_prepare(file_input = "test_mono.wav", N=2048):  #N -size of analysis window

    #load audio
    s, fs = librosa.load("test_mono.wav")

    #output input size
    L = s.size

    # Hop size
    H = int(N/4)

    #output audio size
    Lout = s.size

    # signal blocks for processing and output
    phi  = np.zeros(N)
    out = np.zeros(N, dtype=complex)
    sigout = np.zeros(L)

    # max input amp, window
    win = np.hanning(N)
    p = 0

    while p < L-(N+H):

        # take the spectra of two consecutive windows     
        spec1 =  fft(win*s[p:p+N])
        spec2 =  fft(win*s[p+H:p+N+H])
        # take their phase difference and integrate
        phi += (np.angle(spec2) - np.angle(spec1))
        
        # bring the phase back to between pi and -pi    
        phi =  (( -phi + np.pi) % (2.0 * np.pi ) - np.pi) * -1.0       
        
        out.real, out.imag = np.cos(phi), np.sin(phi)
        
        # inverse FFT and overlap-add        
        sigout[p:p+N] += win*ifft(out*abs(spec2)).real
        
        p += H

    return [fs,s,sigout]

In [ ]:
def disp(fs,inp_s,out_s):
    print('Input audio')
    display(Audio(data=inp_s,rate=fs))
    print('Out audio')
    display(Audio(data=out_s,rate=fs))

In [ ]:
#fs,inp_s,out_s = load_prepare(file_input = "test_mono.wav", N=2048)

In [ ]:
#disp(fs,inp_s,out_s)

## Time scaling

In [ ]:
def time_scale(fs,inp_s,scale = 0.5, N=2048):

    tscale = float(1/scale)
    L = inp_s.size

    # Hop size
    H = int(N/4)

    #output audio size
    Lout = int(L/tscale+N)

    # signal blocks for processing and output
    phi  = np.zeros(N)
    out = np.zeros(N, dtype=complex)
    sigout = np.zeros(Lout)

    # max input amp, window
    win = np.hanning(N)
    pout = 0
    pstretch = 0

    while pstretch < L-(N+H):

        p = int(pstretch)
        
        # take the spectra of two consecutive windows     
        spec1 =  fft(win*inp_s[p:p+N])
        spec2 =  fft(win*inp_s[p+H:p+N+H])
        # take their phase difference and integrate
        phi += (np.angle(spec2) - np.angle(spec1))
        
        # bring the phase back to between pi and -pi    
        phi =  (( -phi + np.pi) % (2.0 * np.pi ) - np.pi) * -1.0       
        
        out.real, out.imag = np.cos(phi), np.sin(phi)
        
        # inverse FFT and overlap-add        
        sigout[pout:pout+N] += win*ifft(out*abs(spec2)).real
        
        pout += H
        pstretch += H*tscale

    return [fs,inp_s,sigout]

In [ ]:
if __name__ == '__main__':
    [inputFile, outputFile, ratio] = sys.argv[0:]

    fs,inp_s,out_s = load_prepare(file_input = "test_mono.wav", N=2048)
    fs_1, orig_f_1, new_f_1 = time_scale(fs,inp_s,scale = 0.5, N=2048)
    sf.write("test_momo_r05.wav", new_f_1, fs_1, subtype='PCM_24')
    fs_2, orig_f_2, new_f_2 = time_scale(fs,inp_s,scale = 2, N=2048)
    sf.write("test_momo_r2.wav", new_f_2, fs_2, subtype='PCM_24')

In [ ]:
#fs_1, orig_f_1, new_f_1 = time_scale(fs,inp_s,scale = 0.5, N=2048)

In [ ]:
#disp(fs_1, orig_f_1, new_f_1)

In [ ]:
#fs_2, orig_f_2, new_f_2 = time_scale(fs,inp_s,scale = 2, N=2048)

In [ ]:
#disp(fs_2, orig_f_2, new_f_2)